# Building Machine Learning Data Table: Locks & Weather
# _Factors affecting vessel delays at US Army Corps of Engineers-managed locks on US waterways._

## Springboard Data Science Career Track, Capstone 2
***
* Author: Heather A. Passmore
* Data: United States Army Corps of Engineers, Lock Performance Management System
* Capstone Description: [Project Proposal](https://github.com/PassMoreHeat/springboard/blob/master/Capstone_2/ProjectProposalCap2.md)
* Initial Data Approach: [Data Wrangling Report](https://github.com/PassMoreHeat/springboard/blob/master/Capstone_2/Data_Wrangling_Report.md)
* Milestone Report: [Predicting Vessel Delays for Corps of Engineers-Lock Performance Management System (LPMS) Data](https://github.com/PassMoreHeat/springboard/blob/master/Capstone_2/LPMS_Milestone_Report.md)
* Final Report: In Progress
***
## Table of Contents
1. Import Dataframes from Previous Step & Lock Locations
2. Reformat & Merge to Build Focal Lock Locations and Names
3. Read In NOAA Weather Data & Merge Stations with Lock Identifiers 
4. Merge 10 Focal Lock DataFrame with Daily Weather Data on E_R_L

## 1. Import Dataframes from Previous Step & Lock Locations

In [1]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
import pickle
import scipy.stats
from scipy.stats import norm
import scipy as sp

# Figures inline, visualization style
%matplotlib inline

# Setup Seaborn
sns.set_style("whitegrid")
sns.set_context("talk")

In [2]:
# Bring ML_traf_reduced dataframe into workspace
# load the pickled ML_traf_reduced DataFrame from hard drive in 'rb' mode
with open('/Users/pgagnon/Documents/Springboard/springboard/Capstone_2/LockData/ML_traf_reduced.pickle','rb') as f:
    ML_traf_reduced = pickle.load(f)

In [3]:
# check dataframe
ML_traf_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15538 entries, 4 to 60078
Data columns (total 47 columns):
E_R_L                   15538 non-null object
YYMMDD                  15538 non-null datetime64[ns]
OPS_ID                  15538 non-null object
LOCKAGE_TYPE            15538 non-null object
VESSEL_FUNCTION_TYPE    15537 non-null object
ARRIVAL_DATE            15538 non-null datetime64[ns]
ASST_CODE               105 non-null object
LINE_SEQ_NO             15538 non-null int64
entry_time              8745 non-null float64
exit_time               8747 non-null float64
delay_time              15538 non-null float64
delay                   15538 non-null bool
hhmmss                  15538 non-null object
pct75_delay_time        15538 non-null float64
A                       854 non-null float64
AA                      854 non-null float64
B                       854 non-null float64
BB                      854 non-null float64
C                       854 non-null float64
CC       

In [4]:
# Read in the lock_locations file
lock_locations = pd.read_csv('/Users/pgagnon/Documents/Springboard/springboard/Capstone_2/LockData/Lock_Locations.csv', 
                    index_col=None, header = 0)

In [5]:
# Read in the RIVERMSTR file
rivermstr = pd.read_csv('/Users/pgagnon/Documents/Springboard/springboard/Capstone_2/LockData/RIVERMSTR.csv', 
                    index_col=None, header = 0)

In [6]:
# Read in the LOCKMSTER file
lockmstr = pd.read_csv('/Users/pgagnon/Documents/Springboard/springboard/Capstone_2/LockData/LOCKMSTR.csv', 
                    index_col=None, header = 0)

## 2. Reformat & Merge to Build Focal Lock Locations and Names

In [7]:
# Create 'E_R_L' identifier column in 'lock_locations'
lock_locations['E_R_L'] = lock_locations['EROC'] + '_' + lock_locations['RIVER_CODE'] + '_' + lock_locations['LOCK_NO'].astype(str)
lock_locations.head(2)

# Create 'E_R_L' identifier column in 'lockmstr'
lockmstr['E_R_L'] = lockmstr['EROC'] + '_' + lockmstr['RIVER_CODE'] + '_' + lockmstr['LOCK_NO'].astype(str)
lock_name = lockmstr[['LOCK_NAME', 'E_R_L']]

In [8]:
# Keep only the 10 focal locks in lock_locations: ten_locations
lock_list = ['B2_FB_41', 'B2_GI_3', 'B2_GI_4', 'B2_GI_77', 'B3_MI_27', 'G3_WS_1', 'H2_OH_76', 'M3_GI_11', 
             'M3_GI_12', 'M3_GI_14']
ten_locations = lock_locations[lock_locations['E_R_L'].isin(lock_list)]
ten_locations.head(10)

,EROC,RIVER_CODE,LOCK_NO,LATITUDE,LONGITUDE,E_R_L
17,M3,GI,11,28.683470,-95.972750,M3_GI_11
18,M3,GI,12,28.679456,-95.981652,M3_GI_12
21,M3,GI,14,28.896385,-95.388235,M3_GI_14
40,H2,OH,76,37.932219,-87.373484,H2_OH_76
59,B3,MI,27,38.702169,-90.181615,B3_MI_27
77,B2,GI,77,29.786071,-92.206770,B2_GI_77
81,B2,GI,3,29.965559,-90.026955,B2_GI_3
82,B2,GI,4,29.915097,-89.972151,B2_GI_4
85,B2,FB,41,29.553571,-92.304694,B2_FB_41
163,G3,WS,1,47.665718,-122.397644,G3_WS_1


In [9]:
# Merge ten_locations with rivermstr to add names of Rivers
ten_loc_river = pd.merge(ten_locations, rivermstr, how='left', on='RIVER_CODE')
ten_loc_river.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 7 columns):
EROC          10 non-null object
RIVER_CODE    10 non-null object
LOCK_NO       10 non-null int64
LATITUDE      10 non-null float64
LONGITUDE     10 non-null float64
E_R_L         10 non-null object
RIVER_NAME    10 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 640.0+ bytes


In [10]:
# Merge ten_locations with lockmstr to add names of locks
ten_loc_river_lock = pd.merge(ten_loc_river, lock_name, how='left', on='E_R_L')
ten_loc_river_lock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 8 columns):
EROC          10 non-null object
RIVER_CODE    10 non-null object
LOCK_NO       10 non-null int64
LATITUDE      10 non-null float64
LONGITUDE     10 non-null float64
E_R_L         10 non-null object
RIVER_NAME    10 non-null object
LOCK_NAME     10 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 720.0+ bytes


In [11]:
# Pickle 'ten_locations' for map making
ten_loc_river_lock.to_pickle('ten_loc_river_lock.pickle')

In [12]:
# Make dataframe for reports
focal_locks = ten_loc_river_lock[['E_R_L', 'RIVER_NAME', 'LOCK_NAME', 'LATITUDE', 'LONGITUDE']]
focal_locks

,E_R_L,RIVER_NAME,LOCK_NAME,LATITUDE,LONGITUDE
0,M3_GI_11,GULF INTRACOASTAL WATERWAY,COLORADO RIVER EAST LOCK,28.683470,-95.972750
1,M3_GI_12,GULF INTRACOASTAL WATERWAY,COLORADO RIVER WEST LOCK,28.679456,-95.981652
2,M3_GI_14,GULF INTRACOASTAL WATERWAY,BRAZOS WEST GATE,28.896385,-95.388235
3,H2_OH_76,OHIO RIVER,NEWBURGH LOCK AND DAM,37.932219,-87.373484
4,B3_MI_27,MISSISSIPPI RIVER,CHAIN OF ROCKS LOCK AND DAM 27,38.702169,-90.181615
5,B2_GI_77,GULF INTRACOASTAL WATERWAY,LELAND BOWMAN LOCK,29.786071,-92.206770
6,B2_GI_3,GULF INTRACOASTAL WATERWAY,INNER HRBR NAVIGATION CANL LK,29.965559,-90.026955
7,B2_GI_4,GULF INTRACOASTAL WATERWAY,ALGIERS LOCK,29.915097,-89.972151
8,B2_FB_41,FRESHWATER BAYOU,FRESHWATER BAYOU LOCK,29.553571,-92.304694
9,G3_WS_1,LAKE WASHINGTON SHIP CANAL,HIRAM M CHITTENDEN LOCKS,47.665718,-122.397644


## 3. Read In NOAA Weather Data & Merge Stations with Lock Identifiers

In [13]:
# Import Weather Data Files and Process. Data are from 7 stations, 3 will be used for two locks
# Weather data from : https://www.ncdc.noaa.gov/cdo-web/datatools/findstation
NOAA_daily = pd.read_csv('/Users/pgagnon/Documents/Springboard/springboard/Capstone_2/LockData/NOAA_daily_8_stations.csv', 
                    index_col=None, header = 0, low_memory=False)
NOAA_daily.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,AWND_ATTRIBUTES,PGTM,PGTM_ATTRIBUTES,...,WT17,WT17_ATTRIBUTES,WT18,WT18_ATTRIBUTES,WT19,WT19_ATTRIBUTES,WT21,WT21_ATTRIBUTES,WT22,WT22_ATTRIBUTES
0,USW00024233,"SEATTLE TACOMA INTERNATIONAL AIRPORT, WA US",47.4444,-122.3138,112.8,2013-01-01,6.04,",,X",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00024233,"SEATTLE TACOMA INTERNATIONAL AIRPORT, WA US",47.4444,-122.3138,112.8,2013-01-02,7.16,",,X",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00024233,"SEATTLE TACOMA INTERNATIONAL AIRPORT, WA US",47.4444,-122.3138,112.8,2013-01-03,6.71,",,X",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00024233,"SEATTLE TACOMA INTERNATIONAL AIRPORT, WA US",47.4444,-122.3138,112.8,2013-01-04,6.26,",,X",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00024233,"SEATTLE TACOMA INTERNATIONAL AIRPORT, WA US",47.4444,-122.3138,112.8,2013-01-05,6.93,",,X",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# View Station Names
NOAA_daily.NAME.value_counts()

LAFAYETTE REGIONAL AIRPORT, LA US                   1826
SEATTLE TACOMA INTERNATIONAL AIRPORT, WA US         1826
PALACIOS MUNICIPAL AIRPORT, TX US                   1826
EVANSVILLE REGIONAL AIRPORT, IN US                  1826
ST LOUIS LAMBERT INTERNATIONAL AIRPORT, MO US       1826
NEW ORLEANS AIRPORT, LA US                          1826
ANGLETON LAKE JACKSON BRAZORIA CO AIRPORT, TX US    1820
Name: NAME, dtype: int64

In [15]:
# Get list of columns to remove unneccesary ones
NOAAcols = list(NOAA_daily.columns.values)
print(NOAAcols)

['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND', 'AWND_ATTRIBUTES', 'PGTM', 'PGTM_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES', 'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TAVG', 'TAVG_ATTRIBUTES', 'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'WSF2', 'WSF2_ATTRIBUTES', 'WSF5', 'WSF5_ATTRIBUTES', 'WT01', 'WT01_ATTRIBUTES', 'WT02', 'WT02_ATTRIBUTES', 'WT03', 'WT03_ATTRIBUTES', 'WT04', 'WT04_ATTRIBUTES', 'WT05', 'WT05_ATTRIBUTES', 'WT06', 'WT06_ATTRIBUTES', 'WT07', 'WT07_ATTRIBUTES', 'WT08', 'WT08_ATTRIBUTES', 'WT10', 'WT10_ATTRIBUTES', 'WT11', 'WT11_ATTRIBUTES', 'WT13', 'WT13_ATTRIBUTES', 'WT14', 'WT14_ATTRIBUTES', 'WT15', 'WT15_ATTRIBUTES', 'WT16', 'WT16_ATTRIBUTES', 'WT17', 'WT17_ATTRIBUTES', 'WT18', 'WT18_ATTRIBUTES', 'WT19', 'WT19_ATTRIBUTES', 'WT21', 'WT21_ATTRIBUTES', 'WT22', 'WT22_ATTRIBUTES']


In [16]:
# Explore contents of columns
NOAA_daily.WT22.value_counts()
# to delete:  ALL '_ATTRIBUTES' 

1.0    10
Name: WT22, dtype: int64

In [17]:
# Keep and reorder columns you need
NOAA_daily_clean = NOAA_daily[['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND', 
                               'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 'WSF2',
                               'WSF5', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 
                               'WT07', 'WT08', 'WT10', 'WT11', 'WT13', 'WT14', 'WT15',  'WT16', 'WT17', 
                               'WT18',  'WT19', 'WT21', 'WT22']]
NOAA_daily_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12776 entries, 0 to 12775
Data columns (total 35 columns):
STATION      12776 non-null object
NAME         12776 non-null object
LATITUDE     12776 non-null float64
LONGITUDE    12776 non-null float64
ELEVATION    12776 non-null float64
DATE         12776 non-null object
AWND         12746 non-null float64
PGTM         100 non-null float64
PRCP         12765 non-null float64
SNOW         12048 non-null float64
SNWD         12283 non-null float64
TAVG         6944 non-null float64
TMAX         12757 non-null float64
TMIN         12753 non-null float64
WSF2         12752 non-null float64
WSF5         12648 non-null float64
WT01         5338 non-null float64
WT02         714 non-null float64
WT03         1658 non-null float64
WT04         27 non-null float64
WT05         17 non-null float64
WT06         20 non-null float64
WT07         1 non-null float64
WT08         1399 non-null float64
WT10         4 non-null float64
WT11         2 non-

## Weather data codes:
WT03 - Thunder
WT04 - Ice pellets, sleet, snow pellets, or small hail" 
PRCP - Precipitation
WT05 - Hail (may include small hail)
WT06 - Glaze or rime 
WT07 - Dust, volcanic ash, blowing dust, blowing sand, or blowing obstruction
WT08 - Smoke or haze 
SNWD - Snow depth
WT10 - Tornado, waterspout, or funnel cloud" 
PGTM - Peak gust time
WT11 - High or damaging winds
TMAX - Maximum temperature
WT13 - Mist
WSF2 - Fastest 2-minute wind speed
WSF5 - Fastest 5-second wind speed
SNOW - Snowfall
WT14 - Drizzle
WT15 - Freezing drizzle 
WT16 - Rain (may include freezing rain, drizzle, and freezing drizzle)" 
WT17 - Freezing rain 
WT18 - Snow, snow pellets, snow grains, or ice crystals
WT19 - Unknown source of precipitation 
AWND - Average wind speed
WT21 - Ground fog 
WT22 - Ice fog or freezing fog
WT01 - Fog, ice fog, or freezing fog (may include heavy fog)
WT02 - Heavy fog or heaving freezing fog (not always distinguished from fog)
TAVG - Average Temperature.
TMIN - Minimum temperature

In [18]:
# Convert all WTxx columns to interger and fill NaN with 0
# Code gives warning and there are a lot of columns but this is the idea:
WT_cols = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT10', 'WT11', 'WT13', 'WT14', 'WT15',
           'WT16', 'WT17', 'WT18',  'WT19', 'WT21', 'WT22']
NOAA_daily_clean[WT_cols] = NOAA_daily_clean[WT_cols].fillna(0).astype(int)
NOAA_daily_clean.info()

/Users/pgagnon/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12776 entries, 0 to 12775
Data columns (total 35 columns):
STATION      12776 non-null object
NAME         12776 non-null object
LATITUDE     12776 non-null float64
LONGITUDE    12776 non-null float64
ELEVATION    12776 non-null float64
DATE         12776 non-null object
AWND         12746 non-null float64
PGTM         100 non-null float64
PRCP         12765 non-null float64
SNOW         12048 non-null float64
SNWD         12283 non-null float64
TAVG         6944 non-null float64
TMAX         12757 non-null float64
TMIN         12753 non-null float64
WSF2         12752 non-null float64
WSF5         12648 non-null float64
WT01         12776 non-null int64
WT02         12776 non-null int64
WT03         12776 non-null int64
WT04         12776 non-null int64
WT05         12776 non-null int64
WT06         12776 non-null int64
WT07         12776 non-null int64
WT08         12776 non-null int64
WT10         12776 non-null int64
WT11         12

In [19]:
# Get station code and location in a dataframe
group_by_station = NOAA_daily_clean.groupby(['STATION', 'NAME'])
count_days_by_station = group_by_station.size()
stations = pd.DataFrame(count_days_by_station).reset_index()
stations

,STATION,NAME,0
0,USW00012916,"NEW ORLEANS AIRPORT, LA US",1826
1,USW00012935,"PALACIOS MUNICIPAL AIRPORT, TX US",1826
2,USW00012976,"ANGLETON LAKE JACKSON BRAZORIA CO AIRPORT, TX US",1820
3,USW00013976,"LAFAYETTE REGIONAL AIRPORT, LA US",1826
4,USW00013994,"ST LOUIS LAMBERT INTERNATIONAL AIRPORT, MO US",1826
5,USW00024233,"SEATTLE TACOMA INTERNATIONAL AIRPORT, WA US",1826
6,USW00093817,"EVANSVILLE REGIONAL AIRPORT, IN US",1826


In [20]:
# Create new dataframe of E_R_L and corresponding station
raw_data = {
        'STATION': ['USW00012916', 'USW00012935', 'USW00012976', 'USW00013976', 'USW00013994', 'USW00024233', 'USW00093817'],
        'E_R_L': ['B2_GI_3', 'M3_GI_11', 'M3_GI_14', 'B2_FB_41', 'B3_MI_27', 'G3_WS_1', 'H2_OH_76']}
ERL_station = pd.DataFrame(raw_data, columns = ['STATION','E_R_L'])
station_ERL_look = pd.merge(stations, ERL_station, on='STATION')
station_ERL_lookup = station_ERL_look[['STATION','E_R_L']]
station_ERL_lookup
# Save for NOAA_dup 'B2_GI_4', 'B2_GI_77','M3_GI_12',

,STATION,E_R_L
0,USW00012916,B2_GI_3
1,USW00012935,M3_GI_11
2,USW00012976,M3_GI_14
3,USW00013976,B2_FB_41
4,USW00013994,B3_MI_27
5,USW00024233,G3_WS_1
6,USW00093817,H2_OH_76


In [21]:
# Create SECOND dataframe of E_R_L and corresponding station FOR THE DUPLICATE STATIONS
raw = {
        'STATION': ['USW00012916', 'USW00013976', 'USW00012935'],
        'E_R_L': ['B2_GI_4', 'B2_GI_77','M3_GI_12']}
DUP_station = pd.DataFrame(raw, columns = ['STATION','E_R_L'])
DUP_look = pd.merge(stations, DUP_station, on='STATION')
DUP_lookup = DUP_look[['STATION','E_R_L']]
DUP_lookup

,STATION,E_R_L
0,USW00012916,B2_GI_4
1,USW00012935,M3_GI_12
2,USW00013976,B2_GI_77


In [22]:
# Make copy of NOAA_daily_clean but only with stations that we are using data for at two Locks
# List of weather station locations to drop (Angleton, Evansville, St. Louis, Seattle)
to_drop = ['USW00024233', 'USW00013994', 'USW00093817', 'USW00012976', ]
# query NOAA_daily_clean to keep stations used twice
NOAA_dup = NOAA_daily_clean.query('STATION not in @to_drop')
NOAA_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5478 entries, 3652 to 10949
Data columns (total 35 columns):
STATION      5478 non-null object
NAME         5478 non-null object
LATITUDE     5478 non-null float64
LONGITUDE    5478 non-null float64
ELEVATION    5478 non-null float64
DATE         5478 non-null object
AWND         5477 non-null float64
PGTM         64 non-null float64
PRCP         5470 non-null float64
SNOW         4847 non-null float64
SNWD         5010 non-null float64
TAVG         1736 non-null float64
TMAX         5478 non-null float64
TMIN         5476 non-null float64
WSF2         5477 non-null float64
WSF5         5432 non-null float64
WT01         5478 non-null int64
WT02         5478 non-null int64
WT03         5478 non-null int64
WT04         5478 non-null int64
WT05         5478 non-null int64
WT06         5478 non-null int64
WT07         5478 non-null int64
WT08         5478 non-null int64
WT10         5478 non-null int64
WT11         5478 non-null int64
WT13

In [23]:
# Merge NOAA_daily_clean with station_ERL_lookup: NOAA_seven
NOAA_seven = pd.merge(NOAA_daily_clean, station_ERL_lookup, how='left', left_on='STATION', right_on='STATION')
NOAA_seven.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12776 entries, 0 to 12775
Data columns (total 36 columns):
STATION      12776 non-null object
NAME         12776 non-null object
LATITUDE     12776 non-null float64
LONGITUDE    12776 non-null float64
ELEVATION    12776 non-null float64
DATE         12776 non-null object
AWND         12746 non-null float64
PGTM         100 non-null float64
PRCP         12765 non-null float64
SNOW         12048 non-null float64
SNWD         12283 non-null float64
TAVG         6944 non-null float64
TMAX         12757 non-null float64
TMIN         12753 non-null float64
WSF2         12752 non-null float64
WSF5         12648 non-null float64
WT01         12776 non-null int64
WT02         12776 non-null int64
WT03         12776 non-null int64
WT04         12776 non-null int64
WT05         12776 non-null int64
WT06         12776 non-null int64
WT07         12776 non-null int64
WT08         12776 non-null int64
WT10         12776 non-null int64
WT11         12

In [24]:
# Merge NOAA_dup with DUP_lookup: NOAA_three
NOAA_three = pd.merge(NOAA_dup, DUP_lookup, how='left', left_on='STATION', right_on='STATION')
NOAA_three.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5478 entries, 0 to 5477
Data columns (total 36 columns):
STATION      5478 non-null object
NAME         5478 non-null object
LATITUDE     5478 non-null float64
LONGITUDE    5478 non-null float64
ELEVATION    5478 non-null float64
DATE         5478 non-null object
AWND         5477 non-null float64
PGTM         64 non-null float64
PRCP         5470 non-null float64
SNOW         4847 non-null float64
SNWD         5010 non-null float64
TAVG         1736 non-null float64
TMAX         5478 non-null float64
TMIN         5476 non-null float64
WSF2         5477 non-null float64
WSF5         5432 non-null float64
WT01         5478 non-null int64
WT02         5478 non-null int64
WT03         5478 non-null int64
WT04         5478 non-null int64
WT05         5478 non-null int64
WT06         5478 non-null int64
WT07         5478 non-null int64
WT08         5478 non-null int64
WT10         5478 non-null int64
WT11         5478 non-null int64
WT13    

In [25]:
# Concatenate NOAA_seven and NOAA_three
weather_10_locks = pd.concat([NOAA_seven, NOAA_three])

# Rename DATE to to YYMMDD so that it matches the lock dataframe
weather_10_locks.rename(columns={'DATE':'YYMMDD'}, inplace=True)

# Format YYMMDD column as datetime
weather_10_locks['YYMMDD'] = pd.to_datetime(weather_10_locks['YYMMDD'])
weather_10_locks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18254 entries, 0 to 5477
Data columns (total 36 columns):
STATION      18254 non-null object
NAME         18254 non-null object
LATITUDE     18254 non-null float64
LONGITUDE    18254 non-null float64
ELEVATION    18254 non-null float64
YYMMDD       18254 non-null datetime64[ns]
AWND         18223 non-null float64
PGTM         164 non-null float64
PRCP         18235 non-null float64
SNOW         16895 non-null float64
SNWD         17293 non-null float64
TAVG         8680 non-null float64
TMAX         18235 non-null float64
TMIN         18229 non-null float64
WSF2         18229 non-null float64
WSF5         18080 non-null float64
WT01         18254 non-null int64
WT02         18254 non-null int64
WT03         18254 non-null int64
WT04         18254 non-null int64
WT05         18254 non-null int64
WT06         18254 non-null int64
WT07         18254 non-null int64
WT08         18254 non-null int64
WT10         18254 non-null int64
WT11    

## 4. Merge 10 Focal Lock DataFrame with Daily Weather Data on E_R_L

In [26]:
# Left merge ML_traf_reduced with weather_10_locks on 'E_R_L' & 'YYMMDD': ML_LPMS
ML_LPMS = pd.merge(ML_traf_reduced, weather_10_locks, how='left', left_on=['E_R_L', 'YYMMDD'], right_on = ['E_R_L', 'YYMMDD'])
ML_LPMS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15538 entries, 0 to 15537
Data columns (total 81 columns):
E_R_L                   15538 non-null object
YYMMDD                  15538 non-null datetime64[ns]
OPS_ID                  15538 non-null object
LOCKAGE_TYPE            15538 non-null object
VESSEL_FUNCTION_TYPE    15537 non-null object
ARRIVAL_DATE            15538 non-null datetime64[ns]
ASST_CODE               105 non-null object
LINE_SEQ_NO             15538 non-null int64
entry_time              8745 non-null float64
exit_time               8747 non-null float64
delay_time              15538 non-null float64
delay                   15538 non-null bool
hhmmss                  15538 non-null object
pct75_delay_time        15538 non-null float64
A                       854 non-null float64
AA                      854 non-null float64
B                       854 non-null float64
BB                      854 non-null float64
C                       854 non-null float64
CC       

In [27]:
# Assign the columns created from stall-stoppage REASON_CODE to a list: reason_cols
reason_cols = ['A', 'AA', 'B', 'BB', 'C', 'CC', 'D', 'DD', 'E', 'EE', 'F', 'FF', 'G', 'GG', 'H', 'I', 'J', 'K', 'L', 
               'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [28]:
# Make a summed unplanned stoppage column with max delay per row
# 'unplan_max_stop' is the maximum mean duration (minutes) of an unplanned stall-stoppage on a day at a lock
# In this column the reason code is lost
ML_LPMS['unplan_max_stop'] = ML_LPMS[[ 'A', 'AA', 'B', 'BB', 'C', 'CC', 'D', 'DD', 'E', 'EE', 'F', 'FF', 'G', 'GG', 
                                      'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 
                                      'X', 'Y', 'Z']].max(axis=1).fillna(0)
ML_LPMS.unplan_max_stop.describe()

count     15538.000000
mean         24.049765
std        1165.665675
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      133920.000000
Name: unplan_max_stop, dtype: float64

In [29]:
# Check column sums of all reason codes for unplanned stoppages. Remove zero sum columns
ML_LPMS[[ 'A', 'AA', 'B', 'BB', 'C', 'CC', 'D', 'DD', 'E', 'EE', 'F', 'FF', 'G', 'GG', 
                                      'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 
                                      'X', 'Y', 'Z']].sum()
# Reason Codes columns 'BB', 'C', 'FF', G', 'GG' sum to zero. Deleted below.

A      30667.500000
AA      2117.000000
B        264.000000
BB         0.000000
C          0.000000
CC       335.000000
D        120.000000
DD        94.000000
E       4193.500000
EE     60935.666667
F       1240.500000
FF         0.000000
G          0.000000
GG         0.000000
H        414.333333
I       3902.500000
J       3813.000000
K        370.000000
L       1165.000000
M        572.500000
N       1260.000000
O        835.000000
P        217.000000
Q       1328.000000
R     171264.500000
S       3219.000000
T      44451.000000
U         65.000000
V        135.000000
W         95.000000
X       4631.000000
Y       1109.500000
Z      39455.750000
dtype: float64

In [30]:
# Check column totals of Weather Type columns and remove all columns with 10 or fewer events
# Will delete: 'WT07', 'WT10', 'WT11', 'WT15', 'WT17', 'WT19', 'WT22' (below)
ML_LPMS[WT_cols].sum()

WT01    6462
WT02     888
WT03    2174
WT04      23
WT05      17
WT06      15
WT07       1
WT08    1696
WT10       6
WT11       3
WT13     331
WT14      40
WT15       1
WT16     287
WT17       3
WT18      48
WT19      10
WT21      23
WT22       7
dtype: int64

In [36]:
#Define a generic function using Pandas replace function
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

In [32]:
# Fill empty cells of ASST_CODE with zero
ML_LPMS['ASST_CODE'] = ML_LPMS['ASST_CODE'].fillna(0)
ML_LPMS.ASST_CODE.value_counts()

0    15433
I       96
Z        9
Name: ASST_CODE, dtype: int64

In [33]:
#Coding ASST_CODE as I=1, Z=1:
print('Before Coding:')
print(pd.value_counts(ML_LPMS['ASST_CODE']))
ML_LPMS['assist'] = coding(ML_LPMS['ASST_CODE'], {'I':1,'Z':1})
print('\nAfter Coding:')
print(pd.value_counts(ML_LPMS['assist']))

Before Coding:
0    15433
I       96
Z        9
Name: ASST_CODE, dtype: int64

After Coding:
0    15433
1      105
Name: assist, dtype: int64


In [34]:
# Assign missing VESSEL_FUNCTION_TYPE the most common type, T, tow or tug boat
# count of T = 12656 before
ML_LPMS['VESSEL_FUNCTION_TYPE'] = ML_LPMS['VESSEL_FUNCTION_TYPE'].fillna('T')
ML_LPMS.VESSEL_FUNCTION_TYPE.value_counts()

T    12657
R      870
K      702
C      592
F      441
P      159
G       50
N       30
Z       15
E       14
M        5
U        3
Name: VESSEL_FUNCTION_TYPE, dtype: int64

In [35]:
# Check contents of delay before recoding
ML_LPMS.delay.value_counts()

False    8435
True     7103
Name: delay, dtype: int64

In [38]:
#Coding 'delay' as False=0, True=1:
print('Before Coding:')
print(pd.value_counts(ML_LPMS['delay']))
ML_LPMS['delay_cat'] = coding(ML_LPMS['delay'], {False:0, True:1})
print('\nAfter Coding:')
print(pd.value_counts(ML_LPMS['delay_cat']))

Before Coding:
False    8435
True     7103
Name: delay, dtype: int64

After Coding:
0.0    8435
1.0    7103
Name: delay_cat, dtype: int64


In [39]:
# Assign missing 'entry_time' and 'exit_time' cells with 0
ML_LPMS.update(ML_LPMS[['entry_time', 'exit_time']].fillna(0))

In [40]:
# Assign empty cells in REASON_CODE-based columns 'reason_cols' with 0
ML_LPMS.update(ML_LPMS[reason_cols].fillna(0))

In [41]:
# Assign columns with NOAA weather data (except WT/WeatherType) to a list
weather_cols = ['AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 'WSF2', 'WSF5']

In [42]:
# Assign empty cells in weather columns 'weather_cols' with 0
ML_LPMS.update(ML_LPMS[weather_cols].fillna(0))

In [43]:
# Create categories from ARRIVAL_DATE: year, day_of_year, day_of_week, month, hour_of_day
ML_LPMS['year'] = ML_LPMS['ARRIVAL_DATE'].dt.year
ML_LPMS['day_of_year'] = ML_LPMS['ARRIVAL_DATE'].dt.dayofyear
ML_LPMS['day_of_week'] = ML_LPMS['ARRIVAL_DATE'].dt.dayofweek
ML_LPMS['month'] = ML_LPMS['ARRIVAL_DATE'].dt.month
ML_LPMS['hour_of_day'] = ML_LPMS['ARRIVAL_DATE'].dt.hour

In [44]:
# Get a list of all the ML_LPMS columns
cols = list(ML_LPMS.columns.values)
print(cols)

['E_R_L', 'YYMMDD', 'OPS_ID', 'LOCKAGE_TYPE', 'VESSEL_FUNCTION_TYPE', 'ARRIVAL_DATE', 'ASST_CODE', 'LINE_SEQ_NO', 'entry_time', 'exit_time', 'delay_time', 'delay', 'hhmmss', 'pct75_delay_time', 'A', 'AA', 'B', 'BB', 'C', 'CC', 'D', 'DD', 'E', 'EE', 'F', 'FF', 'G', 'GG', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 'WSF2', 'WSF5', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT10', 'WT11', 'WT13', 'WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WT19', 'WT21', 'WT22', 'unplan_max_stop', 'assist', 'delay_cat', 'year', 'day_of_year', 'day_of_week', 'month', 'hour_of_day']


In [47]:
# Keep only the columns useful for machine learning
# Remove: 'line_seq_no', 'STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION' (not data)
# REMOVE: 'BB', 'C', 'FF', G', 'GG', (all zeros)
# Remove: 'WT07', 'WT10', 'WT11', 'WT15', 'WT17', 'WT19', 'WT22' (too few events) 
# Remove: ASST_CODE (keep new column, 'assist'), 'delay' (keep new column, 'delay_cat')
ML_LPMS = ML_LPMS[['E_R_L', 'YYMMDD', 'delay_cat', 'LOCKAGE_TYPE', 'VESSEL_FUNCTION_TYPE', 'assist', 'year', 
                   'day_of_year', 'day_of_week', 'month', 'hour_of_day', 'entry_time', 'exit_time', 'delay_time', 
                   'pct75_delay_time', 'unplan_max_stop', 'A', 'AA', 'B', 'CC', 'D', 'DD', 'E', 'EE', 'F', 'H', 'I', 
                   'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'AWND', 
                   'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 
                  'WSF2', 'WSF5', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 
                  'WT13', 'WT14', 'WT16', 'WT17', 'WT18', 'WT21']]

In [48]:
# Check that you have the columns you want
ML_LPMS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15538 entries, 0 to 15537
Data columns (total 67 columns):
E_R_L                   15538 non-null object
YYMMDD                  15538 non-null datetime64[ns]
delay_cat               15538 non-null float64
LOCKAGE_TYPE            15538 non-null object
VESSEL_FUNCTION_TYPE    15538 non-null object
assist                  15538 non-null int64
year                    15538 non-null int64
day_of_year             15538 non-null int64
day_of_week             15538 non-null int64
month                   15538 non-null int64
hour_of_day             15538 non-null int64
entry_time              15538 non-null float64
exit_time               15538 non-null float64
delay_time              15538 non-null float64
pct75_delay_time        15538 non-null float64
unplan_max_stop         15538 non-null float64
A                       15538 non-null float64
AA                      15538 non-null float64
B                       15538 non-null float64
CC    

In [49]:
# Check the head
ML_LPMS.head()

,E_R_L,YYMMDD,delay_cat,LOCKAGE_TYPE,VESSEL_FUNCTION_TYPE,assist,year,day_of_year,day_of_week,month,...,WT04,WT05,WT06,WT08,WT13,WT14,WT16,WT17,WT18,WT21
0,M3_GI_12,2015-03-14,1.0,O,T,0,2015,73,5,3,...,0,0,0,0,0,0,0,0,0,0
1,M3_GI_12,2015-09-04,0.0,O,T,0,2015,247,4,9,...,0,0,0,0,0,0,0,0,0,0
2,B2_GI_4,2016-07-11,0.0,S,T,0,2016,193,0,7,...,0,0,0,0,0,0,0,0,0,0
3,M3_GI_14,2017-04-02,0.0,O,T,0,2017,92,6,4,...,0,0,0,0,0,0,0,0,0,0
4,B2_GI_3,2013-12-14,1.0,S,T,0,2013,348,5,12,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ML_LPMS['delay_cat'] = ML_LPMS['delay_cat'].astype(int)

In [50]:
# Pickle ML_LPMS for Machine Learning step
ML_LPMS.to_pickle('ML_LPMS_final.pickle')